In [2]:
import h5py
import numpy as np
import pandas as pd
import scipy as sp
import networkx as nx
if nx.__version__ == '2.8.8':
    import warnings 
    warnings.simplefilter(action='ignore', category=FutureWarning)
from node_classification import node_categories
names_node_cats = ['bulk', 'root', 'dense sprout', 'sparse sprout', 'inner tree node', 'proper leaf']
import copy

In [3]:

def compute_WS_grid_measures(G, A, P, nn, diffs, flows):
    
    loads = np.abs(flows)
    max_load_idx = np.argmax(loads)
    load_idx = list(list(G.edges)[max_load_idx])
    
    # For backup measures, i.e., post failure of maximally loaded line
    G_post = copy.deepcopy(G)
    G_post.remove_edge(*load_idx)
    post_connected = nx.is_connected(G_post)

    df = pd.DataFrame()
    
    # node features
    
    df["node_number"] = nn

    df["P"] = P
    
    df["AP"] = A.dot(P)
    
    df["AAP"] = A.dot(df["AP"])
    
    df["AAAP"] = A.dot(df["AAP"])
    
    df["row_sum_AA"] = A.dot(A.dot(np.ones(len(P))))
    
    df["row_sum_AAA"] = A.dot(df["row_sum_AA"])
    
    df["degree"] = [d[1] for d in G.degree]
    
    df["degree_centrality"] = nx.degree_centrality(G).values()
    
    df["neighbor_degree_min"] = [min(df["degree"][G.neighbors(node)]) for node in G.nodes]
    
    df["neighbor_degree_max"] = [max(df["degree"][G.neighbors(node)]) for node in G.nodes]
    
    df["neighbor_degree_mean"] = [np.array(df["degree"][G.neighbors(node)]).mean() for node in G.nodes]
    
    df["clustering"] = nx.clustering(G).values()
    
    df["betweenness_centrality"] = nx.betweenness_centrality(G).values()
    
    df["closeness_centrality"] = nx.closeness_centrality(G).values()
    
    df["load_centrality"] = nx.load_centrality(G).values()
    
    df["eigenvector_centrality"] = nx.eigenvector_centrality(G,max_iter=10000).values()
    
    df["katz_centrality"] = nx.katz_centrality(G).values()
    
    df["second_order_centrality"] = nx.second_order_centrality(G).values()
    
    df["current_flow_closeness_centrality"] = nx.current_flow_closeness_centrality(G).values()
    
    df["current_flow_betweenness_centrality"] = nx.current_flow_betweenness_centrality(G).values()
    
    df["average_neighbor_degree"] = nx.average_neighbor_degree(G).values()
    
    df["harmonic_centrality"] = nx.harmonic_centrality(G).values()
    
    df["square_clustering"] = nx.square_clustering(G).values()
    
    df["eccentricity"] = nx.eccentricity(G).values()
    
    # This redundant feature is only here for backwards compatibility
    df["node_cat"] = node_categories(G, denseThres=5).values()
    for cat in names_node_cats:
#         ddf[cat] = [node == cat for node in df["node_cat"]] -> df[cat] = [int(node == cat) for node in df["node_cat"]
        df[cat] = [int(node == cat) for node in df["node_cat"]]

        
    # new node features
    
    df["fiedler_vector"] = nx.fiedler_vector(G)
    
    connected_to_max_load_line = np.zeros(len(G), dtype=bool)
    connected_to_max_load_line[load_idx] = True
    df["node_connected_to_max_load_line"] = connected_to_max_load_line
    
    # Assign an index to every edge, 
    # find indices of connected edges for very node (e[2]), 
    # use indices to look up corresponding loads
    nx.set_edge_attributes(G, dict(zip(G.edges, range(len(G.edges())))), "edge_idx")
    df["max_load_connected_lines"] = [loads[[e[2] for e in G.edges(nbunch=node, data="edge_idx")]].max() / 9.0 for node in G.nodes]
    df["min_load_connected_lines"] = [loads[[e[2] for e in G.edges(nbunch=node, data="edge_idx")]].min() / 9.0 for node in G.nodes] 
    df["mean_load_connected_lines"] = [loads[[e[2] for e in G.edges(nbunch=node, data="edge_idx")]].mean() / 9.0 for node in G.nodes] 
    
    # nx.set_edge_attributes(G, dict(zip(G.edges, 9 * np.cos(diffs))), "laplace_weights")
    df["resistance_distance_centrality"] = nx.current_flow_closeness_centrality(G, weight="laplace_weights").values()
    
    # graph features
    
    N = len(G.nodes)
    M = len(G.edges)
    
    df["degree_assortativity_coefficient"]= nx.degree_assortativity_coefficient(G)
    
    df["transitivity"] = nx.transitivity(G)
    
    df["diameter"] = nx.diameter(G)
    
    nx.set_node_attributes(G, dict(zip(G.nodes, P)), "P")
    
    df["attribute_assortativity_coefficient_P"] = nx.attribute_assortativity_coefficient(G, "P")
    
    df["kirchhoff_index"] = np.reciprocal(df["current_flow_closeness_centrality"]).mean()
    
    df["resistance_distance_kirchhoff_index"] = np.reciprocal(df["resistance_distance_centrality"]).mean()
    
    # new graph features
    
    spec = nx.laplacian_spectrum(G)
    
    df["inverse_algebraic_connectivity"] = 1 / spec[1]
    
    df["eigenratio"] = spec[1] / spec[-1]
     
    df["power_sign_ratio"] = (sum([-P[e[0]] * P[e[1]] for e in G.edges]) + M) / (2 * M)
    
    df["maximal_line_load"] = loads[max_load_idx] / 9.0
    
    df["universal_kuramoto_order_parameter"] = np.cos(diffs).sum() / M
    
    df["connected_post"] = post_connected
    if post_connected:
        # Post failure steady state in DC approximation, i.e, P = L theta
        theta_post = sp.sparse.linalg.lsqr(nx.laplacian_matrix(G_post),P)[0]
        theta_post = theta_post - theta_post[0] # choose first theta as 0
        diffs_post = [theta_post[e[0]] - theta_post[e[1]] for e in G_post.edges]
        loads_post = np.abs(9 * np.sin(diffs_post))      
        # Line outage distribution factor (LODF) in DC approximation
        df["maximal_line_load_post_dc"] = loads_post.max() / 9.0
        df["backup_capacity"] = (loads_post - np.delete(loads, max_load_idx)).max()       
    else:
        df["maximal_line_load_post_dc"] = np.nan
        df["backup_capacity"] = np.nan
  
    return df


In [4]:
df_nodes = pd.read_csv("../WS_nodes.csv")
df_edges = pd.read_csv("../WS_edges.csv")

In [5]:
# df_nodes = df_nodes[0:256]
# df_edges = df_edges[0:4*256]

In [6]:
df_n_computed = pd.DataFrame()
for nf in df_nodes.net_file.unique():
    df_e = df_edges[df_edges["net_file"] == nf]
    df_n = df_nodes[df_nodes["net_file"] == nf]
    G = nx.empty_graph(64) # Create nodes first, so node indices appear in order. This is important for indexing into G.nodes for example
    G.add_edges_from(zip(df_e.sources - 1, df_e.destinations -1)) # offset by 1 because Julia vs Python.
    A = nx.adjacency_matrix(G)
    P = df_n.P
    nn = df_n.node_number
    diffs = df_e.src_phi - df_e.dst_phi + df_e.alpha
    flows = df_e.K * np.sin(diffs)
    laplace_weights = df_e.K * np.cos(diffs)
    nx.set_edge_attributes(G, dict(zip(G.edges, laplace_weights)), "laplace_weights")

    df_n_c = compute_WS_grid_measures(G, A, P.values, nn.values, diffs.values, flows.values)
    df_n_c["net_file"] = nf

    df_n_computed = pd.concat([df_n_computed, df_n_c])



In [7]:

df_final = pd.merge(df_nodes, df_n_computed, on=["net_file", "node_number"], how='outer')


In [8]:
df_final.to_csv("WS_data.csv", index=False)